### Federal Reserve Economic Data (FRED)

The Federal Reserve Economic Data (FRED) is a repository curated by the Federal Reserve Bank of St. Louis. FRED provides access to over 765,000 economic data time series. <br><br>
The purpose of this notebook is to show how, using Python to extract and analyse the data, FRED can be an invaluable source of economic data for research. I first explore U.S. Real Median Household Income by State, over the last 20 years. I then use FRED time series T10Y3M (10-Year Treasury Constant Maturity Minus 3-Month Treasury Constant Maturity) and T10Y2Y (10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity) to  analyse the yield spread. Finally, using Catboost, I demonstate how FRED time series might be used to inform a model to forcast the direction of the movement of the S&P 500 over the next day.

In [3]:
# import required packages
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import pandas_profiling as pp 
import datapungi_fed as dpf
#import plotly.graph_objs as go
import plotly
import plotly.express as px
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
from sklearn.model_selection import RepeatedKFold
import plotly.graph_objects as go

### Visualizing FRED geospatial data

Data can be downloaded from FRED in a format suitable for displaying in a shapefile; this allows a FRED time series to be visualized on a map. Below I retrieve data for U.S. Real Median Household Income by State, covering the last 20 years and surface it on a choropleth map of the U.S.
The chart is a plotly chart and as such you can zoom in/out and pan using the toolbox on the upper right (the toolbox becomes visible if you move the mouse to the upper right of the chart). Hovering the mouse over any of the states shows the household income for that state.

I added a slider to the map, which allows you to see the data on a year to year basis, from 2000 to present. We can clearly see that the higher income states are in the North East and on the West coast, while there is a cluster of South Eastern states that consistenly have the lowest income. Analysing this data in detail could provide a wealth of insights; for instance, it is quite clear that Utah's medium household income has risen steadily over the last decade, from USD63,217 in 2011 to USD84,523 in 2019, this information may be important if you were considering investing in a business in Utah or buying property in Utah.

Notes on the FRED Real Median Household Income time series<br>
Source: U.S. Census Bureau  
Release: Income and Poverty in the United States<br> 
Units:  CPI-U-RS Adjusted Dollars, Not Seasonally Adjusted<br>
Frequency:  Annual<br>
Household data are collected as of March<br>
Estimation of Median Incomes.

In [5]:
plotly.offline.init_notebook_mode(connected=True)
# api key.  note: I intend to delete this key so if you copy this code please request a free key from the FRED site so that 
# your code will not suddenly stop working unexpectedly. 
data = dpf.data('39a76a8c8154c31e8243a777ce48430b')

# first year of geo data
start_year = 2010
start_date = datetime.datetime.strptime(str(start_year), '%Y')

# Real Median Household Income
df = data.geo(series_id='MEHOINUSALA672N',start_date=start_date.date())

# plotly built in state choropleth expects a location of a two character state code
df['State']  = df.series_id.apply(lambda x: x[8:10])

df = df.astype('str')

# all fields are of datatype object, want the value field to be numeric so that plotly colouring is continuous
df['value'] = pd.to_numeric(df['value'])

z_max = df['value'].max()
z_min = df['value'].min()

data_slider = []
for year in df['_date'].unique():
    df_segmented =  df[(df['_date']== year)]


    data_each_yr = dict(
                        type='choropleth',
                        locations = df_segmented['State'],
                        z=df_segmented['value'].astype(float),
                        zmax = z_max, 
                        zmin = z_min, 
                        locationmode='USA-states',
                        colorscale = 'spectral',
                        text=df_segmented.apply(lambda row: "State: {} \
                            <br>Household Income: ${:,}" .format(row['region'],int(row['value'])), axis=1),  
                        hoverinfo="text",
                        colorbar= {'title':'Household<br>Income'}
                    )

    data_slider.append(data_each_yr)
       
steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + start_year))
    step['args'][1][i] = True
    steps.append(step)


sliders = [dict(active=len(steps) - 1, pad={"t": 1}, steps=steps)]

layout = dict(title ='U.S. Real Median Household Income', 
                geo=dict(scope='usa',projection={'type': 'albers usa'}),
                sliders=sliders)

fig = go.Figure(
    data=data_slider,
    layout=layout
)

#fig.show()
plotly.offline.iplot(fig)

In [6]:
plotly.offline.init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = 'jupyterlab'
# api key.  note: I intend to delete this key so if you copy this code please request a free key from the FRED site so that 
# your code will not suddenly stop working unexpectedly. 
data = dpf.data('39a76a8c8154c31e8243a777ce48430b')

# first year of geo data
start_year = 2010
start_date = datetime.datetime.strptime(str(start_year), '%Y')

# Real Median Household Income
df = data.geo(series_id='MEHOINUSALA672N',start_date=start_date.date())

# plotly built in state choropleth expects a location of a two character state code
df['State']  = df.series_id.apply(lambda x: x[8:10])

df = df.astype('str')

# all fields are of datatype object, want the value field to be numeric so that plotly colouring is continuous
df['value'] = pd.to_numeric(df['value'])

z_max = df['value'].max()
z_min = df['value'].min()

data_slider = []
for year in df['_date'].unique():
    df_segmented =  df[(df['_date']== year)]


    data_each_yr = dict(
                        type='choropleth',
                        locations = df_segmented['State'],
                        z=df_segmented['value'].astype(float),
                        zmax = z_max, 
                        zmin = z_min, 
                        locationmode='USA-states',
                        colorscale = 'spectral',
                        text=df_segmented.apply(lambda row: "State: {} \
                            <br>Household Income: ${:,}" .format(row['region'],int(row['value'])), axis=1),  
                        hoverinfo="text",
                        colorbar= {'title':'Household<br>Income'}
                    )

    data_slider.append(data_each_yr)
       
steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + start_year))
    step['args'][1][i] = True
    steps.append(step)


sliders = [dict(active=len(steps) - 1, pad={"t": 1}, steps=steps)]

layout = dict(title ='U.S. Real Median Household Income', 
                geo=dict(scope='usa',projection={'type': 'albers usa'}),
                sliders=sliders)

fig = go.Figure(
    data=data_slider,
    layout=layout
)

#fig.show()
plotly.offline.iplot(fig)

In [ ]:
plotly.offline.init_notebook_mode(connected=False)
import plotly.io as pio
pio.renderers.default = 'jupyterlab'

# api key.  note: I intend to delete this key so if you copy this code please request a free key from the FRED site so that 
# your code will not suddenly stop working unexpectedly. 
data = dpf.data('39a76a8c8154c31e8243a777ce48430b')

# first year of geo data
start_year = 2010
start_date = datetime.datetime.strptime(str(start_year), '%Y')

# Real Median Household Income
df = data.geo(series_id='MEHOINUSALA672N',start_date=start_date.date())

# plotly built in state choropleth expects a location of a two character state code
df['State']  = df.series_id.apply(lambda x: x[8:10])

df = df.astype('str')

# all fields are of datatype object, want the value field to be numeric so that plotly colouring is continuous
df['value'] = pd.to_numeric(df['value'])

z_max = df['value'].max()
z_min = df['value'].min()

data_slider = []
for year in df['_date'].unique():
    df_segmented =  df[(df['_date']== year)]


    data_each_yr = dict(
                        type='choropleth',
                        locations = df_segmented['State'],
                        z=df_segmented['value'].astype(float),
                        zmax = z_max, 
                        zmin = z_min, 
                        locationmode='USA-states',
                        colorscale = 'spectral',
                        text=df_segmented.apply(lambda row: "State: {} \
                            <br>Household Income: ${:,}" .format(row['region'],int(row['value'])), axis=1),  
                        hoverinfo="text",
                        colorbar= {'title':'Household<br>Income'}
                    )

    data_slider.append(data_each_yr)
       
steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + start_year))
    step['args'][1][i] = True
    steps.append(step)


sliders = [dict(active=len(steps) - 1, pad={"t": 1}, steps=steps)]

layout = dict(title ='U.S. Real Median Household Income', 
                geo=dict(scope='usa',projection={'type': 'albers usa'}),
                sliders=sliders)

fig = go.Figure(
    data=data_slider,
    layout=layout
)

#fig.show()
plotly.offline.iplot(fig)

In [ ]:
#!jupyter labextension install @jupyterlab/plotly-extension
plotly.offline.init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = 'jupyterlab'

# api key.  note: I intend to delete this key so if you copy this code please request a free key from the FRED site so that 
# your code will not suddenly stop working unexpectedly. 
data = dpf.data('39a76a8c8154c31e8243a777ce48430b')

# first year of geo data
start_year = 2010
start_date = datetime.datetime.strptime(str(start_year), '%Y')

# Real Median Household Income
df = data.geo(series_id='MEHOINUSALA672N',start_date=start_date.date())

# plotly built in state choropleth expects a location of a two character state code
df['State']  = df.series_id.apply(lambda x: x[8:10])

df = df.astype('str')

# all fields are of datatype object, want the value field to be numeric so that plotly colouring is continuous
df['value'] = pd.to_numeric(df['value'])

z_max = df['value'].max()
z_min = df['value'].min()

data_slider = []
for year in df['_date'].unique():
    df_segmented =  df[(df['_date']== year)]


    data_each_yr = dict(
                        type='choropleth',
                        locations = df_segmented['State'],
                        z=df_segmented['value'].astype(float),
                        zmax = z_max, 
                        zmin = z_min, 
                        locationmode='USA-states',
                        colorscale = 'spectral',
                        text=df_segmented.apply(lambda row: "State: {} \
                            <br>Household Income: ${:,}" .format(row['region'],int(row['value'])), axis=1),  
                        hoverinfo="text",
                        colorbar= {'title':'Household<br>Income'}
                    )

    data_slider.append(data_each_yr)
       
steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + start_year))
    step['args'][1][i] = True
    steps.append(step)


sliders = [dict(active=len(steps) - 1, pad={"t": 1}, steps=steps)]

layout = dict(title ='U.S. Real Median Household Income', 
                geo=dict(scope='usa',projection={'type': 'albers usa'}),
                sliders=sliders)

fig = go.Figure(
    data=data_slider,
    layout=layout
)

#fig.show()
plotly.offline.iplot(fig)

In [4]:
#!jupyter labextension install @jupyterlab/plotly-extension
#plotly.offline.init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = 'jupyterlab'

# api key.  note: I intend to delete this key so if you copy this code please request a free key from the FRED site so that 
# your code will not suddenly stop working unexpectedly. 
data = dpf.data('39a76a8c8154c31e8243a777ce48430b')

# first year of geo data
start_year = 2010
start_date = datetime.datetime.strptime(str(start_year), '%Y')

# Real Median Household Income
df = data.geo(series_id='MEHOINUSALA672N',start_date=start_date.date())

# plotly built in state choropleth expects a location of a two character state code
df['State']  = df.series_id.apply(lambda x: x[8:10])

df = df.astype('str')

# all fields are of datatype object, want the value field to be numeric so that plotly colouring is continuous
df['value'] = pd.to_numeric(df['value'])

z_max = df['value'].max()
z_min = df['value'].min()

data_slider = []
for year in df['_date'].unique():
    df_segmented =  df[(df['_date']== year)]


    data_each_yr = dict(
                        type='choropleth',
                        locations = df_segmented['State'],
                        z=df_segmented['value'].astype(float),
                        zmax = z_max, 
                        zmin = z_min, 
                        locationmode='USA-states',
                        colorscale = 'spectral',
                        text=df_segmented.apply(lambda row: "State: {} \
                            <br>Household Income: ${:,}" .format(row['region'],int(row['value'])), axis=1),  
                        hoverinfo="text",
                        colorbar= {'title':'Household<br>Income'}
                    )

    data_slider.append(data_each_yr)
       
steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + start_year))
    step['args'][1][i] = True
    steps.append(step)


sliders = [dict(active=len(steps) - 1, pad={"t": 1}, steps=steps)]

layout = dict(title ='U.S. Real Median Household Income', 
                geo=dict(scope='usa',projection={'type': 'albers usa'}),
                sliders=sliders)

fig = go.Figure(
    data=data_slider,
    layout=layout
)

fig.show()
#plotly.offline.iplot(fig)

### Inverted Yield Curve

A yield curve shows the rates of a number of maturities for a given instrument. Rates for short-term maturities tend to be lower than rates for long-term maturities for the same instrument, as buyers of debt expect to be compensated for the increased risk and the scarifice of liquidity associated with committing funds for a longer term. An inverted yield curve describes a curve where the long-term rates are lower than the short-term rates. The difference in maturity rates is often represent as a spread.

Risk-free instruments are preferred when using spreads as indicators, as they reduce the amount of noise that could affect rates, such as company credit rating or performance. Two popular Treasury spreads used in analysis are the 10-Year Treasury Constant Maturity Minus 3-Month Treasury Constant Maturity and 10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity; below I analyse these two spreads, using the S&P 500 as a proxy for the overall market.

In [ ]:
# read in the time series from FRED
start_date = datetime.datetime(2000, 1, 1)
end_date = datetime.datetime(2021, 1, 1)

# 10-Year Treasury Constant Maturity Minus 3-Month Treasury Constant Maturity (T10Y3M)
# 10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity (T10Y2Y)
indices = ['T10Y3M','T10Y2Y'] 
df = pd.DataFrame(columns=indices, index = pd.date_range(start=start_date, end=end_date))
for i in indices:
    df[i] = web.DataReader(i, 'fred',start= start_date, end=end_date)[i]

df['GSPC'] = web.DataReader('^GSPC','yahoo',start_date,end_date)['Adj Close']

In [ ]:
def plot_two_series(series_rhs, series_rhs_desc, series_rhs_smooth=True
                    ,series_lhs='GSPC', series_lhs_desc='S&P 500',series_lhs_smooth=True
                    ,yeild_chart=False ):
    '''
    plot two time series on same figure
    '''
    fig, ax = plt.subplots(figsize = (9,5))
    plt.grid()
    
    if series_lhs_smooth:
        lhs_x = df[series_lhs].dropna().rolling(window=50, center=True).mean()
    else:
        lhs_x = df[series_lhs].dropna()
    p1, = plt.plot(lhs_x, color = '#1897f2',label = series_lhs_desc)
    ax.set_ylabel(series_lhs_desc, fontsize=10);
    
    ax2 = ax.twinx()
    if series_rhs_smooth:
        rhs_x = df[series_rhs].dropna().rolling(window=50, center=True).mean()
    else:
        rhs_x = df[series_rhs].dropna()
    p2, = plt.plot(rhs_x,color = '#08d195',label = series_rhs_desc)
    ax2.set_ylabel(series_rhs_desc, fontsize=10);
    
    fig.legend(handles=[p1, p2], bbox_to_anchor=(1, .75), loc='upper left', fontsize=10)

    plt.title('{} Vs {}' .format(series_lhs_desc,series_rhs_desc), fontsize=12)
    
    if yeild_chart:
        ax2.axhline(y=0, color="red", alpha=0.5)  
        
        spread_df = pd.DataFrame()
        spread_df['running'] = df[series_rhs].dropna().rolling(window=50, center=True).mean().dropna()
        spread_df['next'] = spread_df['running'].shift(-1)
        neg_spread = spread_df[ spread_df['running'] * spread_df['next'] < 0]

        start_date = ''
        end_date = ''
        
        for i in range(len(neg_spread)):
            if neg_spread.iloc[i,0] >= 0:
                start_date = neg_spread.index[i]
            else:
                end_date = neg_spread.index[i]
    
            if start_date and end_date:
                ax.axvspan(start_date,end_date,color='lightgrey',alpha =0.4)
                start_date = ''
                end_date = ''
        
    plt.show()
    

plot_two_series('T10Y2Y','10YR - 2YR Treasury',yeild_chart=True)

plot_two_series('T10Y3M','10YR - 3MTH Treasury',yeild_chart=True)

The ten-year/two-year Treasury spread is considered one of the most accurate leading indicators of recessions. Since the Fed began publishing figures in 1976, each inversion of the ten-year/two-year Treasury spread has preceded a recession within the following year.<br><br>
Using a rolling window of 50 working days, we see that the ten-year/three-month Treasury spread is slightly more sensitive than the ten-year/two-year, and perhaps a better indicator of the future trend of the S&P 500.  The spreads are leading indicators, however quite a few months can pass between the date the yield curve inverts and a recession, so action does not need to be taken immediately on an inversion; however,this insight can inform an investment managers strategy.<br><br>
Looking at the charts it appears that when the ten-year/three-month reverts back to normal is a good indicator of an impending downturn in the S&P 500. We can compare the yield curve to other time series to see if there is any correlation, for instance you may expect a recession to affect the sale of luxury goods; to test this hypothesis I plot the ten-year/three-month Treasury spread against the Amundi S&P Global Luxury ETF-C EUR, which provides exposure to around 80 major luxury-related securities in the world. We can see that there is indeed a steep decline in the price following the ten-year/three-month Treasury spread reverting back to normal after a yield inversion; while inversions of the ten-year/three-month Treasury spread also seem a promising indicator of a potential decline in the  Amundi S&P Global Luxury ETF, we must be aware that this is based on a small sample of data and so should be considered with caution.

In [ ]:
# Amundi S&P Global Luxury ETF-C EUR
df['GLUX'] = web.DataReader('GLUX.MI','yahoo',start_date,end_date)['Adj Close']

plot_two_series('T10Y3M','10YR - 3MTH Treasury',
                series_lhs='GLUX', series_lhs_desc='Amundi S&P Global Luxury',series_lhs_smooth=False,
                yeild_chart=True)


### Catboost Regression

We can explore FRED data to see if any of the time series hold information that may help predict the direction of the S&P 500 over the next day. To perform this analysis we will use a Catboost regression model; CatBoost is a model for gradient boosting using decision trees. 


For input to the model I selected the following popular FRED time series:

* ICE BofA US High Yield Index Option-Adjusted Spread (BAMLH0A0HYM2)
* 3-Month London Interbank Offered Rate (LIBOR), based on U.S. Dollar (USD3MTD156N)
* U.S. / Euro Foreign Exchange Rate (DEXUSEU)
* 10-Year Treasury Constant Maturity Rate (DGS10)
* Monetary Base; Total (BOGMBASE)
* Consumer Price Index for All Urban Consumers: All Items in U.S. City Average (CPIAUCSL)
* Industrial Production: Total Index (INDPRO)
* Unemployment Rate (UNRATE)
* Gross Domestic Product (GDP)
* 3-Month Treasury Bill: Secondary Market Rate (TB3MS)
* Velocity of M2 Money Stock (M2V)
* Effective Federal Funds Rate (FEDFUNDS)
* Manufacturers' New Orders: Durable Goods (DGORDER)

In [ ]:
time_series = ['BAMLH0A0HYM2','USD3MTD156N','DEXUSEU','DGS10','BOGMBASE','CPIAUCSL',\
               'INDPRO','UNRATE','GDP','TB3MS','M2V', 'FEDFUNDS','DGORDER']

# only select to EOY 2019. The data in 2020 is two volitile to test a simple model
df = pd.DataFrame(columns=time_series, index = pd.date_range(start=start_date, end='2019-12-31'))

for i in time_series:
    df[i] = web.DataReader(i, 'fred',start= start_date, end=end_date)[i]

df['GSPC'] = web.DataReader('^GSPC','yahoo',start_date,end_date)['Adj Close']

df.dropna(how='all',inplace = True)
df.fillna(method='ffill', inplace=True)
df.dropna(how='any',inplace = True)

df

### Exploratory Data Analysis
The pandas_profiling package provides a good analysis to start exploring the data. In particular, the Interactions section - towards the bottom of the report - provides a visualisation of how the columns in the dataset interact with each other and can be used to get a quick feel as to how GSPC interacts with each FRED time series. The Correlation heatmap gives the correlation between the factors, and we can see that the S&P is positively correlated to DGORDER, INDPRO, GDP, CPIAUCSL and BOGMBASE, and negatively correlated to M2V.
<br><br>
In addition, hoving over the High Correlation warning shows the fields that the factor has a high correlation with. For example, hoving over the High Correlation warning for the FEDFUNDS field shows that it has a high correlation with the fields USD3MTD156N and TB3MS, which is to be expected given that all three are short term rates. If our model was sophisticated and had hundreds of factors, then if there is a group of highly correlated factors, to speed up the model we would consider retaining one factor to represent the group and removing the other factors.

In [ ]:
pp.ProfileReport(df, title='Profiling Report', explorative=True)


In [ ]:
# the target label is the next day price of the S&P 500
df['y'] = df['GSPC'].shift(-1)

# drop last row as this will be nan for y
df.drop( df.iloc[-1:].index,inplace=True)

# will test on the last 90 days
df_train = df[:-90]
df_test = df[-90:]

X_train = df_train.iloc[:,:-2]
y_train = df_train['y']

X_test = df_test.iloc[:,:-2]
y_test = df_test['y']

In [ ]:
# build the model, use a GridSearchCV to tune the hyperparameters
model = CatBoostRegressor(verbose=0)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=8)

parameters = {'depth'         : [4,8,10],
              'learning_rate' : [0.05, 0.10],
              'iterations'    : [ 50,100, 120]
             }

grid = GridSearchCV(estimator=model, param_grid = parameters, cv = cv, n_jobs=-1)
grid.fit(X_train, y_train) 

print ('Parameters of the best estimator:', grid.best_params_)

y_hat = grid.predict(X_test)

In [ ]:
def compare_direction(y_test,y_hat,up_movement=True):
    '''
    test the similarity of the directon of y_test and y_hat
    :param pandas.core.series.Series y_test: test movements  
    :param numpy.ndarray y_hat: predicted movements
    :param bool up_movement: True if comparison of up movements 
    '''
    direction = pd.DataFrame()
    direction['test'] = (y_test.iloc[:] < y_test.iloc[:].shift(-1)).values
    direction['yhat'] =  ( pd.Series(y_hat).iloc[:] - pd.Series(y_hat).iloc[:].shift(-1) ) 

    # percentage of up (down) movements predicted
    if up_movement:
        direction = direction[(direction['yhat'] > 0) ]
    else:
        direction = direction[(direction['yhat'] < 0) ]
   
    return direction['test'].sum() / direction.shape[0]

In [ ]:
print('Correctly predicted up movements: {:.1f}%' . format(compare_direction(y_test,y_hat) * 100))
print('Correctly predicted down movements: {:.1f}%' . format(compare_direction(y_test,y_hat,up_movement=False) * 100))

### Factor importance
This simple model correctly predicted around 62% of up movements and 55% of down movements; this is only marginally better than a random walk but indicates that there is potential in using FRED time series in a more sophisticated model.<br><br>
We can extract the factor importance from the model. Of the time series selected for the model, GDP and CPI are important macroeconomic factors in our model, which is intuititive; however we can see that the 3-Month USD LIBOR (USD3MTD156N) and the ICE BofA US High Yield OAS (BAMLH0A0HYM2) also appear to be potential factors in calculating a prediction, this is insight can be used to contribute to a more sophisticated model.<br><br>
note: you can hover over each bar to view the details of that time series.

In [ ]:
features = pd.DataFrame()
features['Time Series'] = time_series
features['Importance'] = grid.best_estimator_.get_feature_importance()
features.sort_values('Importance', ascending=False,inplace=True)


fig = px.bar(features, x='Time Series', y='Importance',title='Feature Importance')

fig.update_layout(hoverlabel=dict(bgcolor="#a7fae9"))
    
fig.show()